In [1]:
from datasets import load_dataset
from dataloader import *
from transformer import *
import pandas as pd
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import ot
import torch
import optuna
from torchmetrics.classification import F1Score

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
def validation(model, iterator, criterion, device):
    # set model into evaluation mode
    model.eval()

    # validation
    # loss, metrics for current epoch
    val_epoch_loss = 0
    val_epoch_accuracy = 0

    labels_val = []
    preds_val = []
    f1_scorer = F1Score(task='binary').to(device)

    with torch.no_grad(): # stop graph
        # batches
        for i, batch in enumerate(iterator):
            src = batch[0] # X
            trg = batch[1] # y
            src, trg = torch.tensor(src).to(device), torch.tensor(trg).to(device) # put to cpu/gpu
            output = model(src)
            y_pred = torch.argmax(output, dim=-1) # logits -> labels
            output_reshape = output.contiguous().view(-1, output.shape[-1])
            trg = trg.to(torch.int64)

            loss = criterion(output_reshape, trg) # calculate loss
            agreements = torch.eq(y_pred, trg)
            accuracy = torch.mean(agreements.double()) # calculate accuracy

            labels_val.append(trg)
            preds_val.append(y_pred)

            val_epoch_loss += loss.item()
            val_epoch_accuracy += accuracy

    # put to numpy
    labels_val = torch.cat(labels_val)
    preds_val = torch.cat(preds_val)
    # return mean loss w.r.t. batches
    return val_epoch_loss / len(iterator), val_epoch_accuracy / len(iterator), f1_scorer(preds_val, labels_val)

def plot_training(history, marker=None):
    # put everything on cpu
    for key, value in history.items():
        history[key] = [element.cpu() if isinstance(element, torch.Tensor) else element for element in value]

    plt.subplots_adjust(left=0.1,
                    bottom=0.01,
                    right=1.5,
                    top=0.6,
                    wspace=0.4,
                    hspace=0.4)

    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'])
    plt.plot(history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.title('Training loss')

    # vertical line for marking best epoch
    if marker is not None:
        y_min = min(history['train_loss'] + history['val_loss'])
        y_max = max(history['train_loss'] + history['val_loss'])
        plt.vlines(x=marker, ymin=y_min, ymax=y_max, color='red')

    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'])
    plt.plot(history['val_acc'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.title('Training metric')

    # vertical line for marking best epoch
    if marker is not None:
        y_min = min(history['train_acc'] + history['val_acc'])
        y_max = max(history['train_acc'] + history['val_acc'])
        plt.vlines(x=marker, ymin=y_min, ymax=y_max, color='red')

    plt.show()

def train_save_best(model, iterator, valid_iter, optimizer, criterion, epoch, clip, device):

    # set model into training mode
    model.train()

    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)

    # save data - init
    history = {'train_loss': [],
               'val_loss': [],
               'train_acc': [],
               'val_acc': [],
               'learning_rate': []}
    best_model = None
    best_model_score = 1e9
    best_model_epoch = 0

    # training
    for e in range(epoch):
        # loss, metrics for current epoch
        epoch_loss = 0
        epoch_acc = 0

        # batches
        for i, batch in enumerate(tqdm(iterator)):
            src = batch[0] # X
            trg = batch[1] # y
            src, trg = torch.tensor(src).to(device), torch.tensor(trg).to(device) # put to cpu/gpu
            optimizer.zero_grad() # reset optimizer
            output = model(src) # predict
            y_pred = torch.argmax(output, dim=-1) # logits -> labels
            output_reshape = output.contiguous().view(-1, output.shape[-1])
            trg = trg.to(torch.int64)
            loss = criterion(output_reshape, trg) # calculate loss
            agreements = torch.eq(y_pred, trg)
            accuracy = torch.mean(agreements.double()) # calculate accuracy
            loss.backward() # backward pass

            epoch_loss += loss.item()
            epoch_acc += accuracy / len(iterator)

            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step() # optimize model

        # validation
        val_loss, val_acc, val_f1 = validation(model, valid_iter, optimizer, criterion, device)
        
        scheduler.step(val_loss)

        # save data
        with torch.no_grad():
            current_lr = optimizer.param_groups[0]['lr']

            for key, value in zip(history.keys(), [epoch_loss / len(iterator), val_loss, epoch_acc, val_acc, current_lr]):
                history[key].append(value)

            # save best model (w.r.t validation loss)
            if val_loss < best_model_score:
                best_model = model.state_dict()
                best_model_score = val_loss
                best_model_epoch = e

        # visualization
        print(f"Epoch: {e + 1}  Train Loss: {epoch_loss / len(iterator):.4f} \
              Validation Loss: {val_loss:.4f} \
              Train acc: {epoch_acc:.4f}, \
              Val acc: {val_acc:.4f}, \
              Learning Rate : {optimizer.param_groups[0]['lr'] :.4f}")

    # print training curve
    plot_training(history, marker=best_model_epoch)

    return history, best_model, best_model_score

def concat_dataloaders(dataloader1, dataloader2):
    concat_x = []
    concat_y = []

    for elem in dataloader1:
        for i in range(len(elem[0])):
            concat_x.append(elem[0][i].tolist())
            concat_y.append(elem[1][i].tolist())
    for elem in dataloader2:
        for i in range(len(elem[0])):
            concat_x.append(elem[0][i].tolist())
            concat_y.append(elem[1][i].tolist())
    
    x_tensor = torch.tensor(concat_x, device=device)
    y_tensor = torch.tensor(concat_y, device=device)
    dataset = torch.utils.data.TensorDataset(x_tensor, y_tensor)
    iter = torch.utils.data.DataLoader(dataset = dataset, batch_size = 512, shuffle = True)

    return iter  


In [4]:
# init
tokenizer = Tokenizer()
loader = DataLoader(tokenize = tokenizer.tokenize)

dataset = load_dataset("sentiment140")
data_train = pd.DataFrame({'text': dataset['train']['text'], 'sentiment' : dataset['train']['sentiment']})
data_test = pd.DataFrame({'text': dataset['test']['text'], 'sentiment' : dataset['test']['sentiment']})

concat = [data_train, data_test]
data = pd.concat(concat, ignore_index=True)
data = data.sample(n = 70000, random_state = 42)
# convert string label to binary (int) label (positive:1, negative:0)
data["sentiment"] = data['sentiment'].apply(lambda x : int(x == 4))
# train, test, val split
train_A, valid_A, test_A = loader.make_dataset(data)

dataset = load_dataset("sst2")

data_train = pd.DataFrame({'text': dataset['train']['sentence'], 'sentiment' : dataset['train']['label']})
data_test = pd.DataFrame({'text': dataset['test']['sentence'], 'sentiment' : dataset['test']['label']})
data_val = pd.DataFrame({'text': dataset['validation']['sentence'], 'sentiment' : dataset['validation']['label']})

concat = [data_train, data_test, data_val]
data = pd.concat(concat, ignore_index=True)
data = data.sample(n = len(data), random_state = 42)
# convert string label to binary (int) label (positive:1, negative:0)
data["sentiment"] = data['sentiment'].apply(lambda x : int(x == 1))
# train, test, val split
train_B, valid_B, test_B = loader.make_dataset(data)

vocab = loader.get_vocab(pd.concat([train_A, train_B], ignore_index=True).iloc[:, 0])

train_iter_A, valid_iter_A, test_iter_A = loader.make_iter(train_A, valid_A, test_A,
                                                     batch_size=512,
                                                     device=device,
                                                     vocab=vocab)

train_iter_B, valid_iter_B, test_iter_B = loader.make_iter(train_B, valid_B, test_B,
                                                     batch_size=512,
                                                     device=device,
                                                     vocab=vocab)

# NLP stuff
pad_idx = vocab['__PAD__']
voc_size = len(vocab)
print("Vocabulary Size : ", voc_size)

dataset initializing start


Found cached dataset sentiment140 (C:/Users/atace/.cache/huggingface/datasets/sentiment140/sentiment140/1.0.0/f81c014152931b776735658d8ae493b181927de002e706c4d5244ecb26376997)
100%|██████████| 2/2 [00:00<00:00, 27.03it/s]


Length of data after first step of preprocessing:  70000
Tokenizing the data...
Length of the data :  70000
1


Found cached dataset sst2 (C:/Users/atace/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
100%|██████████| 3/3 [00:00<00:00, 500.18it/s]


Length of data after first step of preprocessing:  70042
Tokenizing the data...
Length of the data :  70042
1
text         [[CLS], wicked, ##bit, ##ch, it, is, because, ...
sentiment                                                    1
len                                                         15
Name: 1102814, dtype: object


100%|██████████| 7000/7000 [00:00<00:00, 19562.59it/s]


dataset initializing done
text         [[CLS], elegant, ##ly, appointed, [SEP]]
sentiment                                           1
len                                                 5
Name: 60611, dtype: object


100%|██████████| 7005/7005 [00:00<00:00, 18830.77it/s]


dataset initializing done
Vocabulary Size :  20369


In [5]:
# Create a mixed validation and test set
train_iter = concat_dataloaders(train_iter_A, train_iter_B)
valid_iter = concat_dataloaders(valid_iter_A, valid_iter_B)
test_iter = concat_dataloaders(test_iter_A, test_iter_B)

In [6]:
# Creating the embedding matrices we use the embedding B for the fused model since this embedding has learnt from the whole vocab.
embeddingA = torch.load("Models/embeddingA_16_trained_downstream.pt")
embeddingB = torch.load("Models/embeddingB_16_trained_downstream.pt")
embedding = torch.load("Models/embeddingB_16_trained_downstream.pt")

## Load the Models

In [7]:
modelA = TransformerClassifier(src_pad_idx = pad_idx,
                              embedding=embeddingA,
                              enc_voc_size = voc_size,
                              max_len = 256,
                              d_model = 16,
                              ffn_hidden = 32,
                              n_head = 2,
                              n_layers = 1,
                              drop_prob = 0.7,
                              device = device)

modelA.load_state_dict(torch.load("./Models/modelA_sentiment140_256"))
modelA.eval()

TransformerClassifier(
  (encoder): Encoder(
    (emb): TransformerEmbedding(
      (tok_emb): TokenEmbedding(
        (embedding): Embedding(20369, 16)
      )
      (pos_emb): PositionalEncoding()
      (drop_out): Dropout(p=0.7, inplace=False)
    )
    (layers): ModuleList(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (attention): ScaleDotProductAttention(
            (softmax): Softmax(dim=-1)
          )
          (w_q): Linear(in_features=16, out_features=16, bias=True)
          (w_k): Linear(in_features=16, out_features=16, bias=True)
          (w_v): Linear(in_features=16, out_features=16, bias=True)
          (w_concat): Linear(in_features=16, out_features=16, bias=True)
        )
        (norm1): LayerNorm()
        (dropout1): Dropout(p=0.7, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=16, out_features=32, bias=True)
          (linear2): Linear(in_features=32, out_features=16, bias=True)
     

In [8]:
modelB = TransformerClassifier(src_pad_idx = pad_idx,
                              embedding=embeddingB,
                              enc_voc_size = voc_size,
                              max_len = 256,
                              d_model = 16,
                              ffn_hidden = 32,
                              n_head = 2,
                              n_layers = 1,
                              drop_prob = 0.7,
                              device = device)

modelB.load_state_dict(torch.load("./Models/modelB_sst2_256"))
modelB.eval()

TransformerClassifier(
  (encoder): Encoder(
    (emb): TransformerEmbedding(
      (tok_emb): TokenEmbedding(
        (embedding): Embedding(20369, 16)
      )
      (pos_emb): PositionalEncoding()
      (drop_out): Dropout(p=0.7, inplace=False)
    )
    (layers): ModuleList(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (attention): ScaleDotProductAttention(
            (softmax): Softmax(dim=-1)
          )
          (w_q): Linear(in_features=16, out_features=16, bias=True)
          (w_k): Linear(in_features=16, out_features=16, bias=True)
          (w_v): Linear(in_features=16, out_features=16, bias=True)
          (w_concat): Linear(in_features=16, out_features=16, bias=True)
        )
        (norm1): LayerNorm()
        (dropout1): Dropout(p=0.7, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=16, out_features=32, bias=True)
          (linear2): Linear(in_features=32, out_features=16, bias=True)
     

## OT Functions

In [9]:
def getSupport(model, trainloader, l, alignment = "acts", numOfBatches= 10):
    '''
    Get the support matrices using Activation-based ("acts") or Weight-based ("wts") alignment 
    '''
    if alignment == "acts":
        activation = None
        for i, data in enumerate(trainloader, 0):
            if i >= numOfBatches:
                break
            
            inputs, targets = data
            outputs = model(inputs)

            if activation is None:
                activation = model.actMatrix[l]
            else:
                activation = torch.cat((activation, model.actMatrix[l]))

        return activation
    elif alignment == "wts":
        return model.state_dict()[l]


In [10]:
def fusion(nameA, nameB, weightA, weightB, train_iter_sentiment140, train_iter_sst2, transport_matrix, beta, a):
    support_y = getSupport(modelB, train_iter_sst2, nameB, alignment="wts")
    # Get the weights at layer "idx" from the first model
    W_A = weightA
    W_B = weightB
    # Align the weights from the first model
    aligned_W = torch.matmul(W_A, torch.matmul(transport_matrix, torch.diag(1 / beta)))
    # Get the X-Support
    n = W_A.shape[0]
    alpha = torch.ones(n) * (1/n)
    support_x = getSupport(modelA, train_iter_sentiment140, nameA, alignment="wts")
    # Calculate the euclidean distance between the supports
    distance = ot.dist(support_x, support_y)
    # Calculate beta
    m = W_B.shape[0]
    beta = torch.ones(m) * (1/m)
    # Calculate the transport matrix using optimal transport
    transport_matrix = torch.from_numpy(ot.emd(alpha.numpy(), beta.numpy(), distance.detach().numpy())).float().reshape((n, m))
    # Align model neurons
    aligned_model = torch.matmul(torch.diag(1 / beta), torch.matmul(transport_matrix.T, aligned_W))
    # Get the weights at layer "idx" from the second model
    fused = (a * aligned_model + (1 - a) * W_B)
    return  fused, transport_matrix, beta

In [11]:
def fusion_multihead(nameA, nameB, weightA, weightB, transport_matrix, beta, head):
    support_y = weightB
    support_x = weightA
    # Get the weights at layer "idx" from the first model
    W_A = weightA
    W_B = weightB
    # Initialize the fused model and transport matrix
    fused = torch.empty(W_B.shape)
    transport_matrix_new = torch.zeros((weightA.shape[0], weightB.shape[0]))
    stride = weightB.shape[0] // head
    for i in range(0, weightB.shape[0], stride):
        # Align the weights from the first model
        aligned_W = torch.matmul(W_A[i:i+stride, :], torch.matmul(transport_matrix, torch.diag(1 / beta)))
        # Get the X-Support
        n = W_A.shape[0] // head
        alpha = torch.ones(n) * (1/n)
        # Calculate the euclidean distance between the supports
        distance = ot.dist(support_x[i:i+stride, :], support_y[i:i+stride, :])
        # Calculate beta
        m = W_B.shape[0] // head
        beta_new = torch.ones(m) * (1/m)
        # Calculate the transport matrix using optimal transport
        transport_matrix_new[i:i+stride, i:i+stride] = torch.from_numpy(ot.emd(alpha.numpy(), beta_new.numpy(), distance.detach().numpy())).float().reshape((n, m))
        # Align model neurons
        aligned_model = torch.matmul(torch.diag(1 / beta_new), torch.matmul(transport_matrix_new[i:i+stride, i:i+stride].T, aligned_W))
        # Get the weights at layer "idx" from the second model
        fused[i:i+stride, :] = (aligned_model + W_B[i:i+stride, :]) / 2 
    return  fused, transport_matrix_new, beta_new

In [12]:
def fusion_crossmultihead(nameA, nameB, weightA, weightB, train_iter_sentiment140, train_iter_sst2, transport_matrix, beta, head):
    W_A_head = weightA.view(head, -1)
    W_B_head = weightB.view(head, -1)
    
    m = W_B_head.shape[1]
    beta_head = torch.ones(m) * (1/m)
    transport_matrix_head = torch.matmul(torch.diag(beta_head), torch.eye(m))

    support_y = getSupport(modelB, train_iter_sst2, nameB, alignment="wts")
    support_x = getSupport(modelA, train_iter_sentiment140, nameA, alignment="wts")

    aligned_W = torch.matmul(W_A_head, torch.matmul(transport_matrix_head, torch.diag(1 / beta_head)))

    dist_head = ot.dist(support_x.view(head, -1), support_y.view(head, -1))

    n = W_A_head.shape[0]
    alpha_head = torch.ones(n) * (1/n)

    m = W_B_head.shape[0]
    beta_head = torch.ones(m) * (1/m)

    transport_matrix_new = torch.from_numpy(ot.emd(alpha_head.numpy(), beta_head.numpy(), dist_head.detach().numpy())).float().reshape((n, m))

    aligned_W_A = torch.matmul(torch.diag(1 / beta_head), torch.matmul(transport_matrix_new.T, aligned_W))
    aligned_W_A = aligned_W_A.view(weightA.shape)
    return fusion_multihead(nameA, nameB, aligned_W_A, weightB, transport_matrix, beta, head)

## Fusion via Optimal Transport

In [13]:
fusedModel = TransformerClassifier(src_pad_idx = pad_idx,
                              embedding = embedding,
                              enc_voc_size = voc_size,
                              max_len = 256,
                              d_model = 16,
                              ffn_hidden = 32,
                              n_head = 2,
                              n_layers = 1,
                              drop_prob = 0.2,
                              device = device)

## Method 1

In [14]:
def obj(trial):
    a = trial.suggest_float('a', 0, 1)
    
    # Create the fused weights matrix
    W_fusion = dict.fromkeys(list(modelA.state_dict().keys()))
    # Initialize the algorithm
    m = list(modelB.state_dict().items())[1][1].shape[1]
    beta = torch.ones(m) * (1/m)
    transport_matrix = torch.matmul(torch.diag(beta), torch.eye(m))

    # Fusion via Optimal Transport
    for (nameA, weightA), (nameB, weightB) in zip(modelA.named_parameters(), modelB.named_parameters()):
        if nameA == "encoder.emb.tok_emb.embedding.weight":
            W_fusion[nameA] = weightA
        else:
            if "weight" in nameA:
                if "encoder" in nameA:
                    if "concat" not in nameA and "linear" not in nameA: 
                        W_fusion[nameA], transport_matrix_triplet, _ = fusion(nameA, nameB, weightA, weightB, train_iter_A, train_iter_B, transport_matrix, beta, a)
                    else:
                        W_fusion[nameA], transport_matrix, beta = fusion(nameA, nameB, weightA, weightB, train_iter_A, train_iter_B, transport_matrix, beta, a)

                else:
                    W_fusion[nameA] = a * weightA + (1-a) * weightB
            elif "bias" in nameA:
                if "encoder" in nameA: 
                    if "concat" not in nameA and "linear" not in nameA: 
                        m = weightB.shape[0]
                        beta_bias = torch.ones(m) * (1/m)
                        W_A_bias = weightA.reshape(m, 1)
                        aligned_bias = torch.matmul(torch.diag(1 / beta_bias), torch.matmul(transport_matrix_triplet.T, W_A_bias))
                        aligned_bias = aligned_bias.reshape(m)
                        W_fusion[nameA] = (aligned_bias + weightB) / 2
                    else:
                        m = weightB.shape[0]
                        beta_bias = torch.ones(m) * (1/m)
                        W_A_bias = weightA.reshape(m, 1)
                        aligned_bias = torch.matmul(torch.diag(1 / beta_bias), torch.matmul(transport_matrix.T, W_A_bias))
                        aligned_bias = aligned_bias.reshape(m)
                        W_fusion[nameA] = (aligned_bias + weightB) / 2
                else:
                    W_fusion[nameA] = a * weightA + (1-a) * weightB
            else:
                W_fusion[nameA] = a * weightA + (1-a) * weightB

    # Assign the weights
    with torch.no_grad():
        for name, param in fusedModel.named_parameters():
            param.data = torch.nn.Parameter(W_fusion[name])

    # Validate the fused model
    criterion = torch.nn.CrossEntropyLoss()
    val_loss, val_acc, val_f1 = validation(fusedModel, valid_iter, criterion, device)
    return val_loss

In [15]:
study = optuna.create_study()
study.optimize(obj, n_trials=20)

[I 2023-01-16 11:30:26,014] A new study created in memory with name: no-name-1572e613-f4a6-4d9c-a1fc-45c20932abe2
C:\Users\atace\AppData\Local\Temp/ipykernel_21784/2549446300.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src, trg = torch.tensor(src).to(device), torch.tensor(trg).to(device) # put to cpu/gpu
[I 2023-01-16 11:30:55,031] Trial 0 finished with value: 0.7823393302304404 and parameters: {'a': 0.5082627878576365}. Best is trial 0 with value: 0.7823393302304404.
[I 2023-01-16 11:31:23,387] Trial 1 finished with value: 0.918323448726109 and parameters: {'a': 0.12506235112925312}. Best is trial 0 with value: 0.7823393302304404.
[I 2023-01-16 11:31:51,491] Trial 2 finished with value: 0.9524152747222355 and parameters: {'a': 0.07653535702242442}. Best is trial 0 with value: 0.7823393302304404.
[I 2023-01-16 11:32:24,251] Tri

In [16]:
a = study.best_params["a"]

# Create the fused weights matrix
W_fusion = dict.fromkeys(list(modelA.state_dict().keys()))
# Initialize the algorithm
m = list(modelB.state_dict().items())[1][1].shape[1]
beta = torch.ones(m) * (1/m)
transport_matrix = torch.matmul(torch.diag(beta), torch.eye(m))
# Fusion via Optimal Transport
for (nameA, weightA), (nameB, weightB) in zip(modelA.named_parameters(), modelB.named_parameters()):
    if nameA == "encoder.emb.tok_emb.embedding.weight":
        W_fusion[nameA] = weightA
    else:
        if "weight" in nameA:
            if "encoder" in nameA:
                if "concat" not in nameA and "linear" not in nameA: 
                    W_fusion[nameA], transport_matrix_triplet, _ = fusion(nameA, nameB, weightA, weightB, train_iter_A, train_iter_B, transport_matrix, beta, a)
                else:
                    W_fusion[nameA], transport_matrix, beta = fusion(nameA, nameB, weightA, weightB, train_iter_A, train_iter_B, transport_matrix, beta, a)
            else:
                W_fusion[nameA] = a * weightA + (1-a) * weightB
        elif "bias" in nameA:
            if "encoder" in nameA: 
                if "concat" not in nameA and "linear" not in nameA: 
                    m = weightB.shape[0]
                    beta_bias = torch.ones(m) * (1/m)
                    W_A_bias = weightA.reshape(m, 1)
                    aligned_bias = torch.matmul(torch.diag(1 / beta_bias), torch.matmul(transport_matrix_triplet.T, W_A_bias))
                    aligned_bias = aligned_bias.reshape(m)
                    W_fusion[nameA] = (aligned_bias + weightB) / 2
                else:
                    m = weightB.shape[0]
                    beta_bias = torch.ones(m) * (1/m)
                    W_A_bias = weightA.reshape(m, 1)
                    aligned_bias = torch.matmul(torch.diag(1 / beta_bias), torch.matmul(transport_matrix.T, W_A_bias))
                    aligned_bias = aligned_bias.reshape(m)
                    W_fusion[nameA] = (aligned_bias + weightB) / 2
            else:
                W_fusion[nameA] = a * weightA + (1-a) * weightB
        else:
            W_fusion[nameA] = a * weightA + (1-a) * weightB
# Assign the weights
with torch.no_grad():
    for name, param in fusedModel.named_parameters():
        param.data = torch.nn.Parameter(W_fusion[name])
# Validate the fused model
criterion = torch.nn.CrossEntropyLoss()
val_loss, val_acc, val_f1 = validation(fusedModel, valid_iter, criterion, device)
print("Validation Loss:     ", val_loss)
print("Validation Accuracy:     ", val_acc.item())
print("Validation F1:     ", val_f1)

C:\Users\atace\AppData\Local\Temp/ipykernel_21784/2549446300.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src, trg = torch.tensor(src).to(device), torch.tensor(trg).to(device) # put to cpu/gpu


Validation Loss:      0.7766070876802716
Validation Accuracy:      0.6497163318452381
Validation F1:      tensor(0.6493)


## Recalibrate the Normalization layers

In [17]:
optimizer = torch.optim.Adam(fusedModel.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

clip = 1

fusedModel.train()
for name, param in fusedModel.named_parameters():
    if "weight" in name or "bias" in name:
        param.requires_grad = False
    
for i, batch in enumerate(tqdm(train_iter)):
            src = batch[0] # X
            trg = batch[1] # y
            src, trg = torch.tensor(src).to(device), torch.tensor(trg).to(device) # put to cpu/gpu
            optimizer.zero_grad() # reset optimizer
            output = fusedModel(src) # predict
            y_pred = torch.argmax(output, dim=-1) # logits -> labels
            output_reshape = output.contiguous().view(-1, output.shape[-1])
            trg = trg.to(torch.int64)
            loss = criterion(output_reshape, trg) # calculate loss
            agreements = torch.eq(y_pred, trg)
            accuracy = torch.mean(agreements.double()) # calculate accuracy
            loss.backward() # backward pass

            torch.nn.utils.clip_grad_norm_(fusedModel.parameters(), clip)
            optimizer.step() # optimize model

  0%|          | 0/219 [00:00<?, ?it/s]C:\Users\atace\AppData\Local\Temp/ipykernel_21784/1843898165.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src, trg = torch.tensor(src).to(device), torch.tensor(trg).to(device) # put to cpu/gpu
100%|██████████| 219/219 [05:38<00:00,  1.55s/it]


In [18]:
# Validate the fused model
criterion = torch.nn.CrossEntropyLoss()
val_loss, val_acc, val_f1 = validation(fusedModel, valid_iter, criterion, device)
print("Validation Loss:     ", val_loss)
print("Validation Accuracy:     ", val_acc.item())
print("Validation F1:     ", val_f1)

C:\Users\atace\AppData\Local\Temp/ipykernel_21784/2549446300.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src, trg = torch.tensor(src).to(device), torch.tensor(trg).to(device) # put to cpu/gpu


Validation Loss:      0.6135832348040172
Validation Accuracy:      0.6727523561507937
Validation F1:      tensor(0.6740)


In [19]:
for name, param in fusedModel.named_parameters():
    param.requires_grad = True

## Train the last Layer

In [20]:
optimizer = torch.optim.Adam(fusedModel.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

clip = 1

fusedModel.train()
for name, param in fusedModel.named_parameters():
    if "encoder" in name:
        param.requires_grad = False
    
for i, batch in enumerate(tqdm(train_iter)):
            src = batch[0] # X
            trg = batch[1] # y
            src, trg = torch.tensor(src).to(device), torch.tensor(trg).to(device) # put to cpu/gpu
            optimizer.zero_grad() # reset optimizer
            output = fusedModel(src) # predict
            y_pred = torch.argmax(output, dim=-1) # logits -> labels
            output_reshape = output.contiguous().view(-1, output.shape[-1])
            trg = trg.to(torch.int64)
            loss = criterion(output_reshape, trg) # calculate loss
            agreements = torch.eq(y_pred, trg)
            accuracy = torch.mean(agreements.double()) # calculate accuracy
            loss.backward() # backward pass

            torch.nn.utils.clip_grad_norm_(fusedModel.parameters(), clip)
            optimizer.step() # optimize model

  0%|          | 0/219 [00:00<?, ?it/s]C:\Users\atace\AppData\Local\Temp/ipykernel_21784/2985598703.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src, trg = torch.tensor(src).to(device), torch.tensor(trg).to(device) # put to cpu/gpu
100%|██████████| 219/219 [05:38<00:00,  1.55s/it]


In [21]:
# Validate the fused model
criterion = torch.nn.CrossEntropyLoss()
val_loss, val_acc, val_f1 = validation(fusedModel, valid_iter, criterion, device)
print("Validation Loss:     ", val_loss)
print("Validation Accuracy:     ", val_acc.item())
print("Validation F1:     ", val_f1)

C:\Users\atace\AppData\Local\Temp/ipykernel_21784/2549446300.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src, trg = torch.tensor(src).to(device), torch.tensor(trg).to(device) # put to cpu/gpu


Validation Loss:      0.6207659414836338
Validation Accuracy:      0.6584418402777777
Validation F1:      tensor(0.6592)


## Test Set

In [25]:
# Test the models on the mixed test set
criterion = torch.nn.CrossEntropyLoss()
test_loss_fused, test_acc_fused, test_f1_fused = validation(fusedModel, test_iter, criterion, device)
criterion = torch.nn.CrossEntropyLoss()
test_loss_A, test_acc_A, test_f1_A = validation(modelA, test_iter, criterion, device)
criterion = torch.nn.CrossEntropyLoss()
test_loss_B, test_acc_B, test_f1_B = validation(modelB, test_iter, criterion, device)

print("Test Loss Fused Model:     ", test_loss_fused)
print("Test Accuracy Fused Model:     ", test_acc_fused.item())
print("Test F1 Fused Model:     ", test_f1_fused)
print("Test Loss Model A:     ", test_loss_A)
print("Test Accuracy Model A:     ", test_acc_A.item())
print("Test F1 Model A:     ", test_f1_A)
print("Test Loss Model B:     ", test_loss_B)
print("Test Accuracy Model B:     ", test_acc_B.item())
print("Test F1 Model B:     ", test_f1_B)

C:\Users\atace\AppData\Local\Temp/ipykernel_21784/2549446300.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src, trg = torch.tensor(src).to(device), torch.tensor(trg).to(device) # put to cpu/gpu


Test Loss Fused Model:      0.620941834790366
Test Accuracy Fused Model:      0.6547924785418312
Test F1 Fused Model:      tensor(0.6543)
Test Loss Model A:      0.653900778719357
Test Accuracy Model A:      0.6947001251726519
Test F1 Model A:      tensor(0.6938)
Test Loss Model B:      0.7619793521506446
Test Accuracy Model B:      0.767161139749408
Test F1 Model B:      tensor(0.7670)


In [26]:
# Test the models on the sentiment140 test set
criterion = torch.nn.CrossEntropyLoss()

test_loss_fused, test_acc_fused, test_f1_fused = validation(fusedModel, test_iter_A, criterion, device)
test_loss_A, test_acc_A, test_f1_A = validation(modelA, test_iter_A, criterion, device)
test_loss_B, test_acc_B, test_f1_B = validation(modelB, test_iter_A, criterion, device)

print("Test Loss Fused Model:     ", test_loss_fused)
print("Test Accuracy Fused Model:     ", test_acc_fused.item())
print("Test F1 Fused Model:     ", test_f1_fused)
print("Test Loss Model A:     ", test_loss_A)
print("Test Accuracy Model A:     ", test_acc_A.item())
print("Test F1 Model A:     ", test_f1_A)
print("Test Loss Model B:     ", test_loss_B)
print("Test Accuracy Model B:     ", test_acc_B.item())
print("Test F1 Model B:     ", test_f1_B)

C:\Users\atace\AppData\Local\Temp/ipykernel_21784/2549446300.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src, trg = torch.tensor(src).to(device), torch.tensor(trg).to(device) # put to cpu/gpu


Test Loss Fused Model:      0.5854056434971946
Test Accuracy Fused Model:      0.6935313278654486
Test F1 Fused Model:      tensor(0.6936)
Test Loss Model A:      0.5179040751286915
Test Accuracy Model A:      0.7646289711378736
Test F1 Model A:      tensor(0.7641)
Test Loss Model B:      1.163452752998897
Test Accuracy Model B:      0.6589266247923588
Test F1 Model B:      tensor(0.6579)


In [27]:
# Test the models on the Stanford Treebank test set
criterion = torch.nn.CrossEntropyLoss()

test_loss_fused, test_acc_fused, test_f1_fused = validation(fusedModel, test_iter_B, criterion, device)
test_loss_A, test_acc_A, test_f1_A = validation(modelA, test_iter_B, criterion, device)
test_loss_B, test_acc_B, test_f1_B = validation(modelB, test_iter_B, criterion, device)

print("Test Loss Fused Model:     ", test_loss_fused)
print("Test Accuracy Fused Model:     ", test_acc_fused.item())
print("Test F1 Fused Model:     ", test_f1_fused)
print("Test Loss Model A:     ", test_loss_A)
print("Test Accuracy Model A:     ", test_acc_A.item())
print("Test F1 Model A:     ", test_f1_A)
print("Test Loss Model B:     ", test_loss_B)
print("Test Accuracy Model B:     ", test_acc_B.item())
print("Test F1 Model B:     ", test_f1_B)

C:\Users\atace\AppData\Local\Temp/ipykernel_21784/2549446300.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src, trg = torch.tensor(src).to(device), torch.tensor(trg).to(device) # put to cpu/gpu


Test Loss Fused Model:      0.6563881507941655
Test Accuracy Fused Model:      0.6153483006293491
Test F1 Fused Model:      tensor(0.6151)
Test Loss Model A:      0.7951585182121822
Test Accuracy Model A:      0.6231140311604585
Test F1 Model A:      tensor(0.6234)
Test Loss Model B:      0.3634702478136335
Test Accuracy Model B:      0.8761044789961113
Test F1 Model B:      tensor(0.8761)
